In [7]:
import random
import pygame
from pygame.locals import *
import copy

PERFECT_SNAKE = [[2,   2**2, 2**3, 2**4],
                [2**8, 2**7, 2**6, 2**5],
                [2**9, 2**10,2**11,2**12],
                [2**16,2**15,2**14,2**13]]


class Bot2048:

    def __init__(self, size):
        self.size = size
        self.game_over = False
        self.board = [[0] * self.size for _ in range(self.size)]
        self.init_board()

        self.tile_colors = {0: (205, 193, 180), 2: (238, 228, 218), 4: (236, 224, 202),
                            8: (242, 177, 121), 16: (245, 149, 99), 32: (246, 124, 95),
                            64: (246, 94, 59), 128: (237, 207, 114), 256: (237, 204, 97),
                            512:(237, 200, 80), 1024: (237, 197, 63), 2048: (237, 194, 46),4096: (217, 94, 46),8192: (27, 4, 46)
                           
                           }

        self.score_font_color = (119, 110, 101)
        self.tile_font_color = (249, 246, 242)
        self.background_color = (187, 173, 160)
        self.grid_color = (205, 193, 180)

        self.tile_width = 100
        self.tile_font_size = 50
        self.score_font_size = 40

        self.tile_padding = 10
        self.border_padding = 20
        self.window_padding = 10

        self.window_width = self.size * self.tile_width + 2 * self.border_padding
        self.window_height = self.size * self.tile_width + 2 * self.border_padding + self.score_font_size

        pygame.init()
        self.window = pygame.display.set_mode((self.window_width, self.window_height))
        pygame.display.set_caption("2048")
        self.clock = pygame.time.Clock()

    def init_board(self):
        self.board = [[0] * self.size for _ in range(self.size)]
        self.add_new_tile()
        self.add_new_tile()

    def add_new_tile(self):
        empty_cells = self.get_empty_cells(self.board)
        if empty_cells:
            row, col = random.choice(empty_cells)
            self.board[row][col] = random.choice([2, 4])

    def get_empty_cells(self, board):
        empty_cells = []
        for i in range(self.size):
            for j in range(self.size):
                if board[i][j] == 0:
                    empty_cells.append((i, j))
        return empty_cells

    def get_new_state(self, board, action):
        new_board = [row[:] for row in board]
        if action == 'left':
            new_board = self.merge_left(new_board)
        elif action == 'right':
            new_board = self.merge_right(new_board)
        elif action == 'up':
            new_board = self.transpose(self.merge_left(self.transpose(new_board)))
        elif action == 'down':
            new_board = self.transpose(self.merge_right(self.transpose(new_board)))
        return new_board, board != new_board

    def merge_left(self, board):
        for i in range(self.size):
            row = board[i]
            row = self.merge_cells(row)
            board[i] = row
        return board

    def merge_right(self, board):
        for i in range(self.size):
            row = board[i]
            row.reverse()
            row = self.merge_cells(row)
            row.reverse()
            board[i] = row
        return board

    def merge_cells(self, row):
        new_row = [0] * self.size
        index = 0
        for i in range(self.size):
            if row[i] == 0:
                continue
            if index == 0:
                new_row[index] = row[i]
                index += 1
            elif new_row[index - 1] == row[i]:
                new_row[index - 1] *= 2
            else:
                new_row[index] = row[i]
                index += 1
        return new_row

    def transpose(self, board):
        return [list(row) for row in zip(*board)]

    def evaluate(self, board):
        score = 0
        for i in range(self.size):
            for j in range(self.size):
                score += board[i][j] * PERFECT_SNAKE[i][j]
        return score

    def check_loss(self, board):
        actions = ['left', 'right', 'up', 'down']
        for action in actions:
            new_board, moved = self.get_new_state(board, action)
            if moved:
                return False
        return True

    def display_board(self):
        self.window.fill(self.background_color)

        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] == 0:
                    tile_color = self.tile_colors[0]
                else:
                    tile_color = self.tile_colors[self.board[i][j]]

                tile_rect = pygame.Rect((j * self.tile_width + self.border_padding, i * self.tile_width + self.border_padding),
                                        (self.tile_width, self.tile_width))
                pygame.draw.rect(self.window, tile_color, tile_rect)

                if self.board[i][j] != 0:
                    self.draw_text(str(self.board[i][j]), self.tile_font_size, self.tile_font_color,
                                    tile_rect.x + self.tile_width/2, tile_rect.y + self.tile_width/2)

        self.draw_text("Score: {}".format(self.evaluate(self.board)), self.score_font_size, self.score_font_color,
                        self.border_padding, self.border_padding + self.size * self.tile_width)

        pygame.display.flip()

    def draw_text(self, text, size, color, x, y):
        font = pygame.font.Font(pygame.font.get_default_font(), size)
        text_surface = font.render(text, True, color)
        text_rect = text_surface.get_rect()
        text_rect.center = (x, y)
        self.window.blit(text_surface, text_rect)

    def play(self):
        pygame.time.set_timer(USEREVENT + 1, 500)

        while not self.game_over:
            for event in pygame.event.get():
                if event.type == USEREVENT + 1:
                    self.window.fill(self.background_color)
                    self.display_board()

                if event.type == QUIT:
                    pygame.quit()

            scores = {}
            actions = ['left', 'right', 'up', 'down']
            for action in actions:
                new_board, moved = self.get_new_state(self.board, action)
                if moved:
                    scores[action] = self.expectiminimax(new_board, 2)[0]

            if scores:
                best_action = max(scores, key=scores.get)
                self.board, moved = self.get_new_state(self.board, best_action)
                if moved:
                    self.add_new_tile()

            if self.check_loss(self.board):
                self.game_over = True

            self.display_board()
            self.clock.tick(60)

    def expectiminimax(self, board, depth, dir=None):
        if self.check_loss(board):
            return -float('inf'), dir
        elif depth < 0:
            return self.evaluate(board), dir

        a = 0
        if depth != int(depth):
            # Player's turn, pick max
            a = -float('inf')
            for direction in ['left', 'right', 'up', 'down']:
                new_board, moved = self.get_new_state(board, direction)
                if moved:
                    res = self.expectiminimax(new_board, depth-0.45, direction)[0]
                    if res > a:
                        a = res
        elif depth == int(depth):
            # Nature's turn, calc average
            a = 0
            open_cells = self.get_empty_cells(board)
            for cell in open_cells:
                new_board = copy.deepcopy(board)
                new_board[cell[0]][cell[1]] = 2
                a += 1.0 / len(open_cells) * self.expectiminimax(new_board, depth - 0.55, dir)[0]

        return a, dir


In [8]:
bot = Bot2048(4)
bot.play()